In [ ]:
!pip install eyed3 -q
!pip install pydub -q
!pip install langdetect -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install pyaudioanalysis langid -q
!pip install SpeechRecognition -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install google-cloud-speech -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.4/289.4 kB 11.0 MB/s eta 0:00:00


In [ ]:
import os
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import ShortTermFeatures
from pydub import AudioSegment
import speech_recognition as sr
from langdetect import detect, DetectorFactory

# Ensuring reproducibility
DetectorFactory.seed = 0

def convert_to_wav(audio_path):
    file_name, file_extension = os.path.splitext(audio_path)
    if file_extension.lower() != ".wav":
        audio = AudioSegment.from_file(audio_path)
        wav_path = f"{file_name}.wav"
        audio.export(wav_path, format="wav")
        return wav_path
    return audio_path

def transcribe_audio(audio_path, language_code='en-US'):
    # Convert to wav if necessary
    wav_path = convert_to_wav(audio_path)

    # Initialize recognizer
    recognizer = sr.Recognizer()

    # Load the audio file
    with sr.AudioFile(wav_path) as source:
        audio_data = recognizer.record(source)

    # Transcribe audio to text
    try:
        text = recognizer.recognize_google(audio_data, language=language_code)
        return text
    except sr.RequestError:
        return "API unavailable"
    except sr.UnknownValueError:
        return "Unable to recognize speech"

def predict_language_from_text(text):
    try:
        lang = detect(text)
        return lang
    except:
        return "Unknown"

def language_code_from_langdetect(lang):
    # Map detected language to language codes used by Google Speech Recognition
    lang_code_map = {
        'en': 'en-US',
        'ta': 'ta-IN',
        # Add other languages as needed
    }
    return lang_code_map.get(lang, 'en-US')

# Path to the audio file
audio_file_path = "/content/test.wav"  # Can be any format supported by pydub

# Step 1: Initial transcription with a general language model
initial_transcription = transcribe_audio(audio_file_path, 'en-US')
print(f"Initial Transcription: {initial_transcription}")

# Step 2: Detect language from the initial transcription
detected_language = predict_language_from_text(initial_transcription)
print(f"Detected Language: {detected_language}")

# Step 3: Get the appropriate language code for the detected language
language_code = language_code_from_langdetect(detected_language)
print(f"Using Language Code: {language_code}")

# Step 4: Transcribe again using the detected language code
final_transcription = transcribe_audio(audio_file_path, language_code)
print(f"Final Transcription: {final_transcription}")

# Optionally, detect language again to confirm
final_detected_language = predict_language_from_text(final_transcription)
print(f"Final Detected Language: {final_detected_language}")


Initial Transcription: you didn't buy that I know the on the Galaxy
Detected Language: en
Using Language Code: en-US
Final Transcription: you didn't buy that I know the on the Galaxy
Final Detected Language: en


In [ ]:
from langdetect import detect, DetectorFactory

# Ensuring reproducibility
DetectorFactory.seed = 0

def detect_language(text):
    try:
        lang = detect(text)
        return lang
    except:
        return "Unknown"

# Example usage
text = "Vanakam en peyar"
detected_language = detect_language(text)
print(f"Detected language: {detected_language}")

# Another example with Tamil text
tamil_text = "இது ஒரு எடுத்துக்காட்டு உர"
tamil_detected_language = detect_language(tamil_text)
print(f"Detected language for Tamil text: {tamil_detected_language}")


Detected language: id
Detected language for Tamil text: ta


In [ ]:
from typing import List
from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech

def transcribe_multiple_languages_v2(
    project_id: str,
    language_codes: List[str],
    audio_file: str,
) -> cloud_speech.RecognizeResponse:
    """Transcribe an audio file in multiple languages."""
    client = SpeechClient()
    lang={}

    with open(audio_file, "rb") as f:
        content = f.read()

    for i in language_codes:
     config = cloud_speech.RecognitionConfig(
        auto_decoding_config=cloud_speech.AutoDetectDecodingConfig(),
        language_codes=[i],  # This should now correctly use all specified languages
        model="latest_long",
    )
     request = cloud_speech.RecognizeRequest(
         recognizer=f"projects/{project_id}/locations/global/recognizers/_",
        config=config,
        content=content,
    )
     response = client.recognize(request=request)
     for j, result in enumerate(response.results):
        lang[i] = result.alternatives[0].confidence




    return lang

# Usage
response = transcribe_multiple_languages_v2(
    "speech-to-speech-hackathon",
    ["en-US", "ta-IN",'fr-FR','ko-KR'],
    "/content/sample-2.mp3"
)
print(response)

{'en-US': 0.8541616797447205}


In [ ]:
print (response)

metadata {
  total_billed_duration {
    seconds: 11
  }
}
results {
  alternatives {
    transcript: "hidden diet"
    confidence: 0.09283457696437836
  }
  result_end_offset {
    seconds: 10
    nanos: 210000000
  }
  language_code: "en-US"
}



In [ ]:
max_key = max(response, key=response.get)
max_key

'ta-IN'

In [ ]:
!apt-get install -y google-cloud-cli
!gcloud auth login
!gcloud auth application-default login

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package google-cloud-cli
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=fiL8X2Yp6jyORCH4Fe9LcCO1hB9rVG&prompt=consent&token_usage=remote&access_type=offline&code_challenge=bhbzNBQFoz-dkPSCiVbX4JjfEZdobs_qsHrKDBLGgMM&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0A

In [ ]:
!gcloud config set project speech-to-speech-hackathon


To update your Application Default Credentials quota project, use the `gcloud auth application-default set-quota-project` command.
Updated property [core/project].


In [ ]:
!gcloud auth application-default set-quota-project speech-to-speech-hackathon


Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "speech-to-speech-hackathon" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.
